# Errors and Crashes

Probably the most important chapter in this section is about how to handle error and crashes. Because at the beginning you will run into a few.

For example:

1. You specified file names or paths that **don't exist**.
2. You try to give an interface a ``string`` as input, where a ``float`` value is expected or you try to specify a parameter that doesn't exist. Be sure to use the right **``input type``** and input name.
3. You wanted to give a list of inputs ``[func1.nii, func2.nii, func3.nii]`` to a node that only expects one input file . **``MapNode``** is your solution.
4. You wanted to run SPM's motion correction on compressed NIfTI files, i.e. ``*.nii.gz``? **SPM** cannot handle that. Nipype's **``Gunzip``** interface can help.
5. You haven't set up all necessary **environment variables**. Nipype for example doesn't find your MATLAB or SPM version.
6. You **forget** to specify a **mandatory input** field.
7. You try to **connect** a node to an input field that another node is **already connected** to.

**Important** note about ``crashfiles``. ``Crashfiles`` are only created when you run a workflow, not during building a workflow. If you have a typo in a folder path, because they didn't happen during runtime, but still during workflow building.

## Example Crash 1: File doesn't exist

When creating a new workflow, very often the initial errors are ``IOError``, meaning Nipype cannot find the right files. For example, let's try to run a workflow on ``sub-11``, that in our dataset doesn't exist.

### Creating the crash

In [ ]:
from nipype import SelectFiles, Node, Workflow
from os.path import abspath as opap
from nipype.interfaces.fsl import MCFLIRT, IsotropicSmooth

# Create SelectFiles node
templates={'func': '{subject_id}/ses-test/func/{subject_id}_ses-test_task-fingerfootlips_bold.nii.gz'}
sf = Node(SelectFiles(templates),
          name='selectfiles')
sf.inputs.base_directory = opap('/data/ds000114')
sf.inputs.subject_id = 'sub-11'

# Create Motion Correction Node
mcflirt = Node(MCFLIRT(mean_vol=True,
                       save_plots=True),
               name='mcflirt')

# Create Smoothing node
smooth = Node(IsotropicSmooth(fwhm=4),
              name='smooth')

# Create a preprocessing workflow
wf = Workflow(name="preprocWF")
wf.base_dir = 'working_dir'

# Connect the three nodes to each other
wf.connect([(sf, mcflirt, [("func", "in_file")]),
            (mcflirt, smooth, [("out_file", "in_file")])])

# Let's the workflow
wf.run()

170822-17:55:18,47 workflow INFO:
	 Workflow preprocWF settings: ['check', 'execution', 'logging']
170822-17:55:18,85 workflow INFO:
	 Running serially.
170822-17:55:18,86 workflow INFO:
	 Executing node selectfiles in dir: /opt/tutorial/notebooks/working_dir/preprocWF/selectfiles
170822-17:55:18,93 workflow ERROR:
	 ['Node selectfiles failed to run on host 054251a8d3e7.']
170822-17:55:18,95 workflow INFO:
	 Saving crash info to /opt/tutorial/notebooks/crash-20170822-175518-jovyan-selectfiles-aa879b1a-b927-4d94-b2ad-32a293dd7953.pklz
170822-17:55:18,96 workflow INFO:
	 Traceback (most recent call last):
  File "/home/jovyan/src/nipype/nipype/pipeline/plugins/linear.py", line 43, in run
    node.run(updatehash=updatehash)
  File "/home/jovyan/src/nipype/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/home/jovyan/src/nipype/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/home/jovyan

RuntimeError: Workflow did not execute cleanly. Check log for details

### Investigating the crash

Hidden, in the log file you can find the relevant information:

    IOError: No files were found matching func template: /data/ds000114/sub-11/ses-test/func/sub-11_ses-test_task-fingerfootlips_bold.nii.gz
    Interface SelectFiles failed to run. 

    170727-23:09:45,323 workflow INFO:
    ***********************************
    170727-23:09:45,324 workflow ERROR:
    could not run node: preprocWF.selectfiles
    170727-23:09:45,325 workflow INFO:
    crashfile: /opt/tutorial/notebooks/crash-20170727-230945-neuro-selectfiles-7cb2b22a-1781-4e8f-87e8-692ad4aa559f.pklz
    
This part tells you that it's an **``IOError``** and that it looked for the file **``/data/ds000114/sub-11/ses-test/func/sub-11_ses-test_task-fingerfootlips_bold.nii.gz``**.

After the line ``***********************************``, you can additional see, that it's the node **``preprocWF.selectfiles``** that crasehd and that you can find a **``crashfile``** to this crash under **``/opt/tutorial/notebooks``**.

### Reading the ``crashfile``

To get the full picture of the error, we can read the content of the ``crashfile`` (that has `pklz` format by default) with the ``bash`` command ``nipypecli crash``. We will get the same information as above, but additionally, we can also see directly the input values of the Node that crashed.

In [ ]:
!nipypecli crash /opt/tutorial/notebooks/crash-*selectfiles-*.pklz



File: /opt/tutorial/notebooks/crash-20170822-175518-jovyan-selectfiles-aa879b1a-b927-4d94-b2ad-32a293dd7953.pklz
Node: preprocWF.selectfiles
Working directory: /opt/tutorial/notebooks/working_dir/preprocWF/selectfiles


Node inputs:

base_directory = /data/ds000114
force_lists = False
ignore_exception = False
raise_on_empty = True
sort_filelist = True
subject_id = sub-11



Traceback: 
Traceback (most recent call last):
  File "/home/jovyan/src/nipype/nipype/pipeline/plugins/linear.py", line 43, in run
    node.run(updatehash=updatehash)
  File "/home/jovyan/src/nipype/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/home/jovyan/src/nipype/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/home/jovyan/src/nipype/nipype/pipeline/engine/nodes.py", line 613, in _run_command
    result = self._interface.run()
  File "/home/jovyan/src/nipype/nipype/interfaces/base.py", line 1085, in run


`nipypecli` allows you to rerun the crashed node using an additional option `-r`.

In [ ]:
!nipypecli crash -r /opt/tutorial/notebooks/crash-*selectfiles-*.pklz



File: /opt/tutorial/notebooks/crash-20170822-175518-jovyan-selectfiles-aa879b1a-b927-4d94-b2ad-32a293dd7953.pklz
Node: preprocWF.selectfiles
Working directory: /opt/tutorial/notebooks/working_dir/preprocWF/selectfiles


Node inputs:

base_directory = /data/ds000114
force_lists = False
ignore_exception = False
raise_on_empty = True
sort_filelist = True
subject_id = sub-11



Traceback: 
Traceback (most recent call last):
  File "/home/jovyan/src/nipype/nipype/pipeline/plugins/linear.py", line 43, in run
    node.run(updatehash=updatehash)
  File "/home/jovyan/src/nipype/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/home/jovyan/src/nipype/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/home/jovyan/src/nipype/nipype/pipeline/engine/nodes.py", line 613, in _run_command
    result = self._interface.run()
  File "/home/jovyan/src/nipype/nipype/interfaces/base.py", line 1085, in run


When running in terminal you can also try options that **enable the Python or Ipython debugger when re-executing: `-d` or `-i`**.

**If you don't want to have an option to rerun the crashed workflow, you can change the format of crashfile to a text format.** You can either change this in a configuration file (you can read more [here](http://nipype.readthedocs.io/en/0.13.1/users/config_file.html#config-file)), or you can directly change the `wf.config` dictionary before running the workflow.

In [ ]:
wf.config['execution']['crashfile_format'] = 'txt'
wf.run()

170730-13:26:04,596 workflow INFO:
	 Workflow preprocWF settings: ['check', 'execution', 'logging']
170730-13:26:04,614 workflow INFO:
	 Running serially.
170730-13:26:04,616 workflow INFO:
	 Executing node selectfiles in dir: /opt/tutorial/notebooks/working_dir/preprocWF/selectfiles
170730-13:26:04,651 workflow ERROR:
	 ['Node selectfiles failed to run on host 4a288ebd42a4.']
170730-13:26:04,654 workflow INFO:
	 Saving crash info to /opt/tutorial/notebooks/crash-20170730-132604-neuro-selectfiles-918286ce-6804-472d-bb30-615eb8d211a8.txt
170730-13:26:04,655 workflow INFO:
	 Traceback (most recent call last):
  File "/opt/conda/envs/neuro/lib/python3.6/site-packages/nipype/pipeline/plugins/linear.py", line 43, in run
    node.run(updatehash=updatehash)
  File "/opt/conda/envs/neuro/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/opt/conda/envs/neuro/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 482, in

RuntimeError: Workflow did not execute cleanly. Check log for details

Now you should have a new text file with your crash report. 

In [ ]:
!cat /opt/tutorial/notebooks/crash-*selectfiles-*.txt

Node: preprocWF.selectfiles
Working directory: /opt/tutorial/notebooks/working_dir/preprocWF/selectfiles

Node inputs:

base_directory = /data/ds000114
force_lists = False
ignore_exception = False
raise_on_empty = True
sort_filelist = True
subject_id = sub-11

Traceback (most recent call last):
  File "/opt/conda/envs/neuro/lib/python3.6/site-packages/nipype/pipeline/plugins/linear.py", line 43, in run
    node.run(updatehash=updatehash)
  File "/opt/conda/envs/neuro/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/opt/conda/envs/neuro/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/opt/conda/envs/neuro/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 613, in _run_command
    result = self._interface.run()
  File "/opt/conda/envs/neuro/lib/python3.6/site-packages/nipype/interfaces/base.py", line 1084, in run


## Example Crash 2: Wrong Input Type or Typo in the parameter

Very simple, if an interface expects a ``float`` as input, but you give it a ``string``, it will crash:

In [ ]:
from nipype.interfaces.fsl import IsotropicSmooth
smooth = IsotropicSmooth(fwhm='4')

TraitError: The 'fwhm' trait of an IsotropicSmoothInput instance must be a float, but a value of '4' <class 'str'> was specified.

This will give you the error: **``TraitError``**``: The 'fwhm' trait of an IsotropicSmoothInput instance must be a float, but a value of '4' <type 'str'> was specified.``

To make sure that you are using the right input types, just check the ``help`` section of a given interface. There you can see **``fwhm: (a float)``**.

In [ ]:
IsotropicSmooth.help()

Wraps command **fslmaths**

Use fslmaths to spatially smooth an image with a gaussian kernel.

Inputs::

	[Mandatory]
	fwhm: (a float)
		fwhm of smoothing kernel [mm]
		flag: -s %.5f, position: 4
		mutually_exclusive: sigma
	in_file: (an existing file name)
		image to operate on
		flag: %s, position: 2
	sigma: (a float)
		sigma of smoothing kernel [mm]
		flag: -s %.5f, position: 4
		mutually_exclusive: fwhm

	[Optional]
	args: (a unicode string)
		Additional parameters to the command
		flag: %s
	environ: (a dictionary with keys which are a bytes or None or a value
		 of class 'str' and with values which are a bytes or None or a value
		 of class 'str', nipype default value: {})
		Environment variables
	ignore_exception: (a boolean, nipype default value: False)
		Print an error message instead of throwing an exception in case the
		interface fails to run
	internal_datatype: ('float' or 'char' or 'int' or 'short' or 'double'
		 or 'input')
		datatype to use for calculations (default is f

In a similar way, you will also get an error message if the input type is correct but you have a type in the name:

    TraitError: The 'output_type' trait of an IsotropicSmoothInput instance must be u'NIFTI_PAIR' or u'NIFTI_PAIR_GZ' or u'NIFTI_GZ' or u'NIFTI', but a value of 'NIFTIiii' <type 'str'> was specified.

In [ ]:
from nipype.interfaces.fsl import IsotropicSmooth
smooth = IsotropicSmooth(output_type='NIFTIiii')

TraitError: The 'output_type' trait of an IsotropicSmoothInput instance must be 'NIFTI' or 'NIFTI_PAIR' or 'NIFTI_GZ' or 'NIFTI_PAIR_GZ', but a value of 'NIFTIiii' <class 'str'> was specified.

## Example Crash 3: Giving an array as input where a single file is expected

As you an see in the [MapNode](basic_mapnodes.ipynb) example, if you try to feed an array as an input into a field that only expects a single file, you will get a **``TraitError``**.

In [ ]:
from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import Node

files = ['/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz',
         '/data/ds000114/sub-02/ses-test/func/sub-02_ses-test_task-fingerfootlips_bold.nii.gz']

gunzip = Node(Gunzip(), name='gunzip',)
gunzip.inputs.in_file = files

TraitError: The 'in_file' trait of a GunzipInputSpec instance must be an existing file name, but a value of ['/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz', '/data/ds000114/sub-02/ses-test/func/sub-02_ses-test_task-fingerfootlips_bold.nii.gz'] <class 'list'> was specified.

This can be solved by using a ``MapNode``:

In [ ]:
from nipype.pipeline.engine import MapNode
gunzip = MapNode(Gunzip(), name='gunzip', iterfield=['in_file'])
gunzip.inputs.in_file = files

Now, make sure that you specify files that actually exist, otherwise you can the same problem as in crash example 1, but this time labeled as ``TraitError``:

    TraitError: Each element of the 'in_file' trait of a DynamicTraitedSpec instance must be an existing file name, but a value of '/data/ds102/sub-06/func/sub-06_task-flanker_run-1_bold.nii.gz' <type 'str'> was specified.

In [ ]:
files = ['/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz',
         '/data/ds000114/sub-03/func/sub-03_task-fingerfootlips_bold.nii.gz']
gunzip.inputs.in_file = files

TraitError: The trait 'in_file' of a DynamicTraitedSpec instance is an existing file name, but the path  '/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz' does not exist.

**By the way, not that those crashes don't create a ``crashfile``, because they didn't happen during runtime, but still during workflow building.**

## Example Crash 4: SPM doesn't like ``*.nii.gz`` files

SPM12 cannot handle compressed NIfTI files (``*nii.gz``). If you try to run the node nonetheless, it can give you different kind of problems:

### SPM Problem 1 with ``*.nii.gz`` files

SPM12 has a problem with handling ``*.nii.gz`` files. For it a compressed functional image has no temporal dimension and therefore seems to be just a 3D file. So if we try to run the ``Realign`` interface on a compressed file, we will get a **``TraitError``** error.

In [ ]:
from nipype.interfaces.spm import Smooth
smooth = Smooth(in_files='/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz')
smooth.run()

TraitError: /data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz is not included in allowed types: .img, .nii, .hdr

### SPM problem 2 with ``*.nii.gz`` files

Sometimes **``TraitError``** can be more misleading.

In [ ]:
from nipype.interfaces.spm import Realign
realign = Realign(in_files='/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz')
realign.run()

TraitError: Each element of the 'in_files' trait of a RealignInputSpec instance must be a list of items which are an existing file name or an existing file name, but a value of '/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz' <class 'str'> was specified.

**This issue can be solved by unzipping the compressed NIfTI file before giving it as an input to an SPM node.** This can either be done by using the ``Gunzip`` interface from Nipype or even better, if the input is coming from a FSL interface, most of them have an input filed `output_type='NIFTI'`, that you can set to NIFIT.

## Example Crash 5: Nipype cannot find the right software

Especially at the beginning, just after installation, you sometimes forgot to specify some environment variables. If you try to use an interface where the environment variables of the software are not specified, e.g. if you try to run:

```python
from nipype.interfaces.freesurfer import MRIConvert
convert = MRIConvert(in_file='/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz',
                     out_type='nii')
```

you migh get an errors, such as:

    IOError: command 'mri_convert' could not be found on host mnotter
    Interface MRIConvert failed to run.

Or if you try to use SPM, but forgot to tell Nipype where to find it. If you forgot to tell the system where to find MATLAB (or MCR), than you will get same kind of error as above. But if you forgot to specify which SPM you want to use, you'll get the following **``RuntimeError``**:

    Standard error:
    MATLAB code threw an exception:
    SPM not in matlab path


You can solve this issue by specifying the path to your SPM version:

```python
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/usr/local/MATLAB/R2017a/toolbox/spm12')
```

## Example Crash 6: You forget mandatory inputs or use input fields that don't exist

One of the simpler errors are the ones connected to input and output fields.

### Forgetting mandatory input fields

Let's see what happens if you forget a **``[Mandatory]``** input field.

In [ ]:
from nipype.interfaces.spm import Realign
realign = Realign(register_to_mean=True)
realign.run()

ValueError: Realign requires a value for input 'in_files'. For a list of required inputs, see Realign.help()

This gives you the error:

    ValueError: Realign requires a value for input 'in_files'. For a list of required inputs, see Realign.help()

As described by the error text, if we use the ``help()`` function, we can actually see, which inputs are mandatory and which are optional.

In [ ]:
realign.help()

Use spm_realign for estimating within modality rigid body alignment

http://www.fil.ion.ucl.ac.uk/spm/doc/manual.pdf#page=25

Examples
--------

>>> import nipype.interfaces.spm as spm
>>> realign = spm.Realign()
>>> realign.inputs.in_files = 'functional.nii'
>>> realign.inputs.register_to_mean = True
>>> realign.run() # doctest: +SKIP

Inputs::

	[Mandatory]
	in_files: (a list of items which are a list of items which are an
		 existing file name or an existing file name)
		list of filenames to realign

	[Optional]
	fwhm: (a floating point number >= 0.0)
		gaussian smoothing kernel width
	ignore_exception: (a boolean, nipype default value: False)
		Print an error message instead of throwing an exception in case the
		interface fails to run
	interp: (0 <= a long integer <= 7)
		degree of b-spline used for interpolation
	jobtype: ('estwrite' or 'estimate' or 'write', nipype default value:
		 estwrite)
		one of: estimate, write, estwrite
	matlab_cmd: (a unicode string)
		matlab command to

### Using input fields that don't exist

Let's see what happens if we try to specify a parameter that doesn't exist as an input field:


In [ ]:
from nipype.interfaces.afni import Despike
despike = Despike(in_file='/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz',
                  output_type='NIFTI')
despike.run()

TraitError: Cannot set the undefined 'output_type' attribute of a 'DespikeInputSpec' object.

This results in the **``TraitError``**:

    TraitError: Cannot set the undefined 'output_type' attribute of a 'DespikeInputSpec' object.

So what went wrong? If you use the ``help()`` function, you will see that the correct input filed is called **``outputtype``** and not **``output_type``**.

## Example Crash 7: Trying to connect a node to an input field that is already occupied

Sometimes when you build a new workflow, you might forget that an output field was already connected and you try to connect a new node to the already occupied field.

First, let's create a simple workflow:

In [ ]:
from nipype import SelectFiles, Node, Workflow
from os.path import abspath as opap
from nipype.interfaces.fsl import MCFLIRT, IsotropicSmooth

# Create SelectFiles node
templates={'func': '{subject_id}/func/{subject_id}_task-fingerfootlips_bold.nii.gz'}
sf = Node(SelectFiles(templates),
          name='selectfiles')
sf.inputs.base_directory = opap('/data/ds000114')
sf.inputs.subject_id = 'sub-01'

# Create Motion Correction Node
mcflirt = Node(MCFLIRT(mean_vol=True,
                       save_plots=True),
               name='mcflirt')

# Create Smoothing node
smooth = Node(IsotropicSmooth(fwhm=4),
              name='smooth')

# Create a preprocessing workflow
wf = Workflow(name="preprocWF")
wf.base_dir = 'working_dir'

# Connect the three nodes to each other
wf.connect([(sf, mcflirt, [("func", "in_file")]),
            (mcflirt, smooth, [("out_file", "in_file")])])

Now, let's create a new node and connect it to the already occupied input field ``in_file`` of the ``smooth`` node:

In [ ]:
# Create a new node
mcflirt_NEW = Node(MCFLIRT(mean_vol=True),
                   name='mcflirt_NEW')

# Connect it to an already connected input field
wf.connect([(mcflirt_NEW, smooth, [("out_file", "in_file")])])

Exception: 
Trying to connect preprocWF.mcflirt_NEW:out_file to preprocWF.smooth:in_file but input 'in_file' of node 'preprocWF.smooth' is already
connected.


This will lead to the error:

```python
Exception: 
Trying to connect preprocWF.mcflirt_NEW:out_file to preprocWF.smooth:in_file but input 'in_file' of node 'preprocWF.smooth' is already connected.
```